# Universidade Estadual de Campinas - Trabalho de Conclusão de Curso 
## Autor - João Velanes 237824

### Parte 1 - Import das bibliotecas a serem utilizadas no projeto 

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import torch.nn.init as init
from torchmetrics import Precision, Recall, Accuracy
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler



In [65]:
import warnings
warnings.filterwarnings('ignore')

In [66]:
df = pd.read_csv("Arquivo Concatenado/falha_2.csv")


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655017 entries, 0 to 655016
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   timestamp   655017 non-null  object 
 1   P-PDG       653476 non-null  float64
 2   P-TPT       545968 non-null  float64
 3   T-TPT       545968 non-null  float64
 4   P-MON-CKP   535750 non-null  float64
 5   T-JUS-CKP   486452 non-null  float64
 6   P-JUS-CKGL  25668 non-null   float64
 7   T-JUS-CKGL  0 non-null       float64
 8   QGL         84414 non-null   float64
 9   class       594031 non-null  float64
dtypes: float64(9), object(1)
memory usage: 50.0+ MB


In [68]:
df["class"].unique()

array([  0., 102.,   2.,  nan])

In [69]:
df1 = df.copy()
df1.drop(columns=["T-JUS-CKGL","timestamp"], inplace=True)
df1.dropna(inplace=True)
df1["class"] = df1["class"].astype(str)
df1["class"].replace({"2.0":"1.0","102.0":"2.0"},inplace=True)
df1["class"] = df1["class"].astype(float)
df1["class"] = df1["class"].astype(int)
print(df1["class"].unique())
#df1.columns

[0 2 1]


In [70]:
df1.columns

Index(['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'P-JUS-CKGL',
       'QGL', 'class'],
      dtype='object')

In [71]:

class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()

        # Assuming the input is a 1D tensor (e.g., [batch_size, 1, 7] if we treat the feature vector as a single channel)
        self.feature_extractor = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1),  # 1D convolution
            nn.ELU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Flatten()
        )

        # Calculate the size after the convolutional layers
        self._to_linear = self._calculate_linear_input_size()

        # Final classifier
        self.classifier = nn.Linear(self._to_linear , num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension: [batch_size, 7] -> [batch_size, 1, 7]
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

    def _calculate_linear_input_size(self):
        # Forward pass with a dummy input to calculate the size
        with torch.no_grad():
            x = torch.randn(1, 7)  # Example input size (batch_size, number_of_features)
            x = x.unsqueeze(1)  # Adding channel dimension to match Conv1d input
            x = self.feature_extractor(x)
            return x.shape[1]



In [72]:
class Classe2(Dataset):
    def __init__(self,data):
        super(Classe2, self).__init__()
        self.data = data.to_numpy(dtype=np.float32) # Usar float 32 por que os pesos da Rede Neural sao Float32
        #display(data)
        
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, idx):
        feature = self.data[idx,:-1]
        label = int(self.data[idx,-1])
        return feature, label

In [73]:
X = df1.iloc[:,:-1]
y = df1.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.15,random_state=42)

In [74]:
y_train.astype(str).value_counts()

class
0    11225
2     7874
1     2599
Name: count, dtype: int64

In [75]:
# Assuming x_train is a pandas DataFrame
scaler = MinMaxScaler()
#scaler = MaxAbsScaler()
#scaler = Normalizer()
#scaler = StandardScaler()
# Fit the scaler and transform the data
x_train_scaled = scaler.fit_transform(x_train)

# Convert the scaled data back to a DataFrame, preserving the column names
x_train_scaled_df = pd.DataFrame(x_train_scaled, columns=x_train.columns)

# Now x_train_scaled_df is a DataFrame with the same columns as x_train
x_train_scaled_df.head()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL
0,0.000000,0.295903,0.140195,0.448614,1.000000,0.001766,0.000000
1,0.000000,0.267060,0.064298,0.282057,1.000000,0.001222,0.000000
2,0.989067,0.076469,0.008978,0.000306,0.008113,0.960915,0.284044
3,0.000000,0.508781,0.911222,0.585144,0.143568,0.001338,0.000000
4,0.000000,1.000000,0.999286,0.999624,1.000000,0.001746,0.000000


In [76]:
# Fazendo o feature scaling para os dados teste
x_test_scaled = scaler.fit_transform(x_test)

# Convert the scaled data back to a DataFrame, preserving the column names
x_test_scaled_df = pd.DataFrame(x_test_scaled, columns=x_test.columns)

# Now x_train_scaled_df is a DataFrame with the same columns as x_train
x_train_scaled_df.head()

,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,P-JUS-CKGL,QGL
0,0.000000,0.295903,0.140195,0.448614,1.000000,0.001766,0.000000
1,0.000000,0.267060,0.064298,0.282057,1.000000,0.001222,0.000000
2,0.989067,0.076469,0.008978,0.000306,0.008113,0.960915,0.284044
3,0.000000,0.508781,0.911222,0.585144,0.143568,0.001338,0.000000
4,0.000000,1.000000,0.999286,0.999624,1.000000,0.001746,0.000000


### Treinamento do Modelo

In [81]:

# Initialize dataset and dataloader
dataset_train = Classe2(pd.concat([x_train_scaled_df.reset_index(drop=True), # Resetar e dropar index para concatenar corretamente 
                                   y_train.reset_index(drop=True)], axis=1))
dataloader_train = DataLoader(dataset_train, batch_size=16, shuffle=True)

# Initialize the neural network, loss function, and optimizer
net = Net(3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001) #0.00001

# Initialize metrics
metric_precision = Precision(task="multiclass", num_classes=3, average="weighted")
metric_recall = Recall(task="multiclass", num_classes=3, average="weighted")

# Loss storage
total_loss = []

# Training loop
num_epochs = 80
for epoch in range(num_epochs):
    epoch_loss = 0.0  # Initialize epoch loss

    # Reset metrics at the beginning of each epoch
    metric_precision.reset()
    metric_recall.reset()

    for features, labels in dataloader_train:
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = net(features)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        epoch_loss += loss.item()  # Accumulate loss

        # Calculate predictions and update metrics
        _, preds = torch.max(outputs, dim=1)
        metric_precision.update(preds, labels)
        metric_recall.update(preds, labels)
        total_loss.append(epoch_loss)
    # Compute average loss for the epoch
    average_loss = epoch_loss / len(dataloader_train)

    # Compute precision and recall for the epoch
    precision = metric_precision.compute()
    recall = metric_recall.compute()

    # Print epoch statistics
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.6f}, Precision: {precision:.4f}, Recall: {recall:.4f}')


Epoch [1/80], Loss: 0.210985, Precision: 0.9245, Recall: 0.9243
Epoch [2/80], Loss: 0.116155, Precision: 0.9610, Recall: 0.9601
Epoch [3/80], Loss: 0.099841, Precision: 0.9660, Recall: 0.9653
Epoch [4/80], Loss: 0.093115, Precision: 0.9700, Recall: 0.9697
Epoch [5/80], Loss: 0.085304, Precision: 0.9726, Recall: 0.9723
Epoch [6/80], Loss: 0.081393, Precision: 0.9743, Recall: 0.9741
Epoch [7/80], Loss: 0.075316, Precision: 0.9766, Recall: 0.9765
Epoch [8/80], Loss: 0.073496, Precision: 0.9758, Recall: 0.9757
Epoch [9/80], Loss: 0.070888, Precision: 0.9781, Recall: 0.9779
Epoch [10/80], Loss: 0.067455, Precision: 0.9779, Recall: 0.9778
Epoch [11/80], Loss: 0.064691, Precision: 0.9803, Recall: 0.9802
Epoch [12/80], Loss: 0.061893, Precision: 0.9797, Recall: 0.9796
Epoch [13/80], Loss: 0.059347, Precision: 0.9806, Recall: 0.9805
Epoch [14/80], Loss: 0.058249, Precision: 0.9810, Recall: 0.9808
Epoch [15/80], Loss: 0.055490, Precision: 0.9817, Recall: 0.9816
Epoch [16/80], Loss: 0.055127, Pre

### Evaluation

In [80]:
import torch
from torchmetrics.classification import Precision, Recall
from torch.utils.data import DataLoader

# Assuming Classe2 and Net are defined elsewhere in your code

# Initialize dataset and dataloader for testing
dataset_test = Classe2(pd.concat([x_test_scaled_df.reset_index(drop=True),
                                  y_test.reset_index(drop=True)], axis=1))
dataloader_test = DataLoader(dataset_test, batch_size=64, shuffle=True)

# Initialize metrics for evaluation
metric_precision = Precision(task="multiclass", num_classes=3, average=None)
metric_recall = Recall(task="multiclass", num_classes=3, average=None)

# Set the network to evaluation mode
net.eval()

all_labels=[]
all_preds=[]
# Disable gradient computation during evaluation
with torch.no_grad():
    for features, labels in dataloader_test:
        outputs = net(features)  # Forward pass
        _, preds = torch.max(outputs, dim=1)  # Get predictions

        # Update the metrics with predictions and true labels
        metric_precision.update(preds, labels)
        metric_recall.update(preds, labels)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
# Compute the final precision and recall after processing all batches
precision = metric_precision.compute()
recall = metric_recall.compute()

# Print the evaluation results
print(f"Precision: {precision}")
print(f"Recall: {recall}")


conf_matrix = confusion_matrix(all_labels, all_preds)
conf_matrix 

Precision: tensor([0.9919, 0.9142, 0.9766])
Recall: tensor([1.0000, 0.9247, 0.9623])


array([[1959,    0,    0],
       [   0,  405,   33],
       [  16,   38, 1379]], dtype=int64)

In [ ]:
sns.set_style("white")

dados_teste = pd.concat([x_test_scaled_df.reset_index(drop=True),y_test.reset_index(drop=True)],axis=1)

sns.pairplot(dados_teste, hue="class")
